In [13]:
from snowflake import connector
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

ctx = connector.connect(
        user="namitha.john@gopuff.com",
        authenticator="externalbrowser",
        role="READER_ROLE",
        warehouse="READER_WH",
        account="gopuff.east-us-2.azure",
    )
cur = ctx.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://gopuff.okta.com/app/snowflake/exk25gqvk9wE1DhXJ4x7/sso/saml?SAMLRequest=nZJPc9owEMW%2Fikc925IdKEUDZEgIrdOkOGCaDDfFFkaxLTlaGUM%2BfWX%2BdNJDcuhNI73d39O%2BHVzuysLZcg1CySHyPYIcLhOVCpkN0TKeut%2BQA4bJlBVK8iHac0CXowGwsqjouDYbOeevNQfj2EYSaPswRLWWVDEQQCUrOVCT0MX4%2Fo4GHqEMgGtjcehUkoKwrI0xFcW4aRqvufCUznBACMGkj62qlXxB7xDV54xKK6MSVZxLdvZPHyB8TDotwiosIToVXgl5HMFnlOejCOiPOI7caLaIkTM%2B%2F%2B5aSahLrhdcb0XCl%2FO7owGwDr7PouV06nEGxq3BDTz2VmvugVTNumA5T1RZ1ca29uwJr3mKC5UJO7BwMkRVLtK3OHx49h%2FnN6%2BPpLl42eXzqxXT0c%2BnfXcFs4x0Xu5vO7msHliYIOf3Od6gjTcEqHko21CNvSJBxyU9N%2BjHxKdBl%2FpfvR7pr5AzsaEKycyh8uw8U1W9XnsqN%2BzgjVUV%2Fmsb810edLPXbd5vbvzJ5um2s%2BthAIXbyNBxa%2BiBr0f%2FNYsBft%2FitIW%2FbDDhJFKFSPbOVOmSmY9z8z3%2FcCNSd32QUl4yUYzTVHMAm19RqOZac2bsshtdc4RHR%2Bq

In [44]:
#bring in all product/location intersections in expiry risk table sorted by Expiry Risk Cost in descending order
SQL = '''
with expiry as (select LOCATION_ID, PRODUCT_ID,SUM(EXPIRY_RISK_QUANTITY), SUM(EXPIRY_RISK_COST), MAX(RECENT_CASE_PACK) from dbt_buffet.core.inventory_expiration_risk 
WHERE NOT LOCATION_NAME='NJ_DC_1179' AND NOT LOCATION_NAME='CA_DC_1486' AND EXPIRY_RISK_COST IS NOT NULL
GROUP BY 1,2)
   , inventory_rec as (SELECT PRODUCT_ID, LOCATION_ID, MAX(EVENT_TIME_OPS_DATE) from   dbt_prod.core.inventory_reconciliation
    where event_state in ('damaged', 'expired')
    GROUP BY 1,2)
select *
from expiry
        join inventory_rec
                   on expiry.LOCATION_ID = inventory_rec.LOCATION_ID AND
                      expiry.PRODUCT_ID = inventory_rec.PRODUCT_ID
    ORDER BY 2,1
;
'''

og_expiry = cur.execute(SQL).fetch_pandas_all()
print(og_expiry)
expiry = og_expiry.to_numpy()
final_report = []
all_rc = [0] * 5

#cycle through every product location intersection in expiry table
for e in range(1,2):
    #data array will contain all information need to find root cause
    data = []
    cutoff_dates = []
    locs = []
#for e in range(2,4):
    #loc_ID = expiry[e][0]
    p_ID = expiry[e][1]
    #loc_ID = 675
    #p_ID = 651444
    #print(e)
    #print(p_ID)
    while expiry[e][1] == p_ID and e < len(expiry):
        data.append([expiry[e][0],p_ID,expiry[e][2],expiry[e][4]])        
        cutoff_dates.append(expiry[e][7])
        locs.append(expiry[e][0])
        print("cutoff:")
        print(expiry[e][7])
        e += 1
    e =- 1
    
    current_loc = 0
    current_loc_ind = -1

    print("data:")
    print(data)
    print(locs)
    #call in inventory reconciliation table for product/location intersection
    SQL_2 = f'''
    with expiry as (select distinct location_id,
                                product_id
                from dbt_buffet.core.inventory_expiration_risk
                where expiry_risk_cost is not null)
   , inventory_rec as (select product_id,
                              location_id,
                              action,
                              event_state,
                              event_time_ops_date,
                              signed_quantity
                       from dbt_prod.core.inventory_reconciliation)
select inventory_rec.*
from inventory_rec
         join expiry
              on expiry.location_id = inventory_rec.location_id AND
                 expiry.product_id = inventory_rec.product_id
where true
  and inventory_rec.location_id not in (1179, 1486)
  and event_state in ('packed', 'discovered', 'lost', 'shelved', 'damage', 'expired')
  and event_time_ops_date >= current_date - interval '2 years'
  and inventory_rec.product_id = {p_ID}
order by location_id, event_time_ops_date'''

    inventory_rec = cur.execute(SQL_2).fetch_pandas_all()
    inventory_rec = inventory_rec.to_numpy()
    print(inventory_rec)

    if len(inventory_rec) != 0:

        #number of recieves in timeframe
        num_recieves = 0

        #list of weekly sell rate in timeframe
        sell_rate = []

        #variables used to track week partitioning in data
        week_sell_rate = 0.0
        prev_week_number = 0
        prev_year_number = 0
        prev_date = inventory_rec[0][4]

        #six week average including current week
        avg_sell_rate = 0.0

        #total_quantity at any point of time with product (possibly not needed)
        total_quantity = 0

        #instance of vendor damage with date of occurance
        vendor=[]

        #instances of off-shelf damage with date of occurance
        shelf = []
    print(locs)

    #cycling through all instances of inventory rec
    for product in inventory_rec:
        if current_loc == 0:
            current_loc = product[1]
            current_loc_ind = 0
            #print("first loc")
            #print(product[1])
            while locs[current_loc_ind] != current_loc and current_loc_ind < len(locs)-1:
                #loc_ID,p_ID,expiry[e][2],expiry[e][4],num_recieves,sell_rate,total_quantity,vendor
                data[current_loc_ind].extend([locs[current_loc_ind], p_ID,expiry[e][2],expiry[e][4], 0,[],0,[]])
                current_loc_ind = current_loc_ind + 1

        elif current_loc != product[1]:
            #print(current_loc_ind)
            data[current_loc_ind].extend([current_loc,p_ID,expiry[e][2],expiry[e][4],num_recieves,sell_rate,total_quantity,vendor])
            current_loc = product[1]
            #print("after loc")
            #print(current_loc_ind)
            while current_loc_ind < len(locs)-1 and locs[current_loc_ind] != current_loc:
                print(current_loc_ind)
                #loc_ID,p_ID,expiry[e][2],expiry[e][4],num_recieves,sell_rate,total_quantity,vendor
                data[current_loc_ind].extend([locs[current_loc_ind], p_ID,expiry[e][2],expiry[e][4], 0,[],0,[]])
                current_loc_ind = current_loc_ind + 1
            num_recieves = 0

            #list of weekly sell rate in timeframe
            sell_rate = []

            #variables used to track week partitioning in data
            week_sell_rate = 0.0
            prev_week_number = 0
            prev_year_number = 0
            prev_date = inventory_rec[0][4]

            #six week average including current week
            avg_sell_rate = 0.0

            #total_quantity at any point of time with product (possibly not needed)
            total_quantity = 0

            #instance of vendor damage with date of occurance
            vendor=[]

        #add signed quanitity to track total_quantity
        total_quantity += product[5]
        event= product[3]
        date = product[4]

        #shelved is when product is recieved 
        if  event=='shelved':
            num_recieves = num_recieves + 1
        
        #packed is when product is sold
        elif event== 'packed':
            # d is week of date of packed event
            d = int(date.strftime("%U"))
            #case of current event is within current week
            if(d == prev_week_number and date.isocalendar().year == prev_year_number):
                week_sell_rate -= product[5]

            #for case of first instance of packed event
            elif (prev_week_number == 0):
                week_sell_rate -= product[5]
                prev_week_number = d
                prev_year_number = date.isocalendar().year
                prev_date = date

            #in case of different week then current tracked week
            else:
                i = len(sell_rate)-1
                #calc what to divide average by in cause of less then 6 weeks looked at 
                div = 6

                #calc avg sell rate
                if i < 4:
                    div = len(sell_rate) +1
                    for w in sell_rate:
                        avg_sell_rate += w[1]
                else:
                    for count in range(5):
                        avg_sell_rate += sell_rate[i][1]
                        i-= 1
                avg_sell_rate = (avg_sell_rate + week_sell_rate)/div

                #add to sell rate list [date of week, sell rate of week, avg of sell rate, current total quantity] of old current week
                sell_rate.append([prev_date,week_sell_rate,avg_sell_rate, total_quantity])

                #case of adding all weeks-- to implement
                #sell_rate[-1][1] = week_sell_rate/7
                #prev_week_number+= 1
                #while prev_week_number != d:
                    #week_date =datetime.strptime(str(prev_year_number)+ " " + str(prev_week_number) + ' 0', "%Y %W %w")
                    #sell_rate.append((week_date,0,avg_sell_rate,total_quantity))
                    #prev_week_number+=1
                
                #set up for new current week to look at
                week_sell_rate =  0 - product[5]
                prev_week_number=d
                prev_year_number = date.isocalendar().year
                prev_date=date

            #if case of a damage event
        elif event=='damaged':
            #before 2023 and in Warehouses, product is recieved to shelf before being damaged so it should be removed from total_quantity count 
            if date.isocalendar().year < 2023 or ("DC" in str(product[1])):
                total_quantity = total_quantity - product[5]
            
            #in case of vendor damage out
            if "vendor" in product[2]:
                vendor.append((date,product[5]))
                #print(vendor)
            
    #data will have location ID, Product ID, expiry risk quantity, case pack size, num of recieves, list of weekly sell rates, total_quantity, vendor damage, off-shelf damage
    #data.append([loc_ID,p_ID,expiry[e][2],expiry[e][4],num_recieves,sell_rate,total_quantity,vendor,shelf])
    print("data:")
    print(data[0])

    #call in replenishment data for product ID and location ID
    SQL_repl = f'''
    with expiry as (select distinct location_id,
                                product_id
                from dbt_buffet.core.inventory_expiration_risk
                where expiry_risk_cost is not null)
   , repl as (select RECOMMENDATION_CREATION_DATE,FINAL_RECOMMENDATION,LOCATION_ID,PRODUCT_ID,ORDER_DATE,NEED_BY_DATE, BUY_QUANTITY_EACHES from dbt_buffet.core.o9_replenishments_recommendations
    where (final_recommendation=TRUE)
    ORDER BY LOCATION_ID,ORDER_DATE)
select repl.*
from repl
         join expiry
              on expiry.location_id = repl.location_id AND
                 expiry.product_id = repl.product_id
where true
  and repl.location_id not in (1179, 1486)
  and order_date >= current_date - interval '2 years'
  and repl.product_id = {p_ID}
order by location_id, order_date'''

    #SQL call for forecasting data (raw data)
    SQL_fore = f'''
    with expiry as (select distinct location_id,
                                product_id
                from dbt_buffet.core.inventory_expiration_risk
                where expiry_risk_cost is not null)
   , fore as (select LOCATION_ID, PRODUCT_ID,FORECASTED_DATE,MODEL_FORECAST,FINAL_FORECAST, DS_FORECAST_STARTED_AT from dbt_prod.core.supply_plan_forecast
    ORDER BY LOCATION_ID,FORECASTED_DATE, DS_FORECAST_STARTED_AT DESC)
select fore.*
from fore
         join expiry
              on expiry.location_id = fore.location_id AND
                 expiry.product_id = fore.product_id
where true
  and fore.location_id not in (1179, 1486)
  and ds_forecast_started_at >= current_date - interval '2 years'
  and fore.product_id = {p_ID}
order by location_id, forecasted_date, ds_forecast_started_at DESC'''

    #SQL call for purchasing data (data for purchasing orders)
    SQL_purch = f'''
    with expiry as (select distinct location_id,
                                product_id
                from dbt_buffet.core.inventory_expiration_risk
                where expiry_risk_cost is not null)
   , purch as (select GP_ITEM_ID, SHIP_TO_GP_LOCATION_ID, NEED_BY_DATE, LBI_QTY_RECEIVED, CLOSED_FOR_RECEIVING_AT from dbt_prod.core.purchase_order_lines
    ORDER BY NEED_BY_DATE)
select purch.*
from purch
         join expiry
              on expiry.location_id = purch.ship_to_gp_location_id AND
                 expiry.product_id = purch.gp_item_id
where true
  and purch.ship_to_gp_location_id not in (1179, 1486)
  and need_by_date >= current_date - interval '2 years'
  and purch.gp_item_id = {p_ID}
order by location_id, need_by_date
    '''

    #SQL call for lbi transitions data (for expiration dates)
    SQL_lbi = f'''with expiry as (select distinct location_id,
                                product_id
                from dbt_buffet.core.inventory_expiration_risk
                where expiry_risk_cost is not null)
   , lbi as (select PRODUCT_ID,LOCATION_ID,EVENT_TIME_OPS_DATE, ACTION, DELTA, STATE, EXPIRATION_DATE from dbt_prod.core.lbi_transitions
    WHERE action in ('receive', 'receive_to_shelf')
    and state in ('shelved', 'received')
    ORDER BY LOCATION_ID, EVENT_TIME_OPS_DATE, EXPIRATION_DATE)
select lbi.*
from lbi
         join expiry
              on expiry.location_id = lbi.location_id AND
                 expiry.product_id = lbi.product_id
where true
  and lbi.location_id not in (1179, 1486)
  and event_time_ops_date >= current_date - interval '2 years'
  and lbi.product_id = {p_ID}
order by location_id, event_time_ops_date, expiration_date'''

    replenishment = cur.execute(SQL_repl).fetch_pandas_all()
    forecast = cur.execute(SQL_fore).fetch_pandas_all()
    purchase_orders = cur.execute(SQL_purch).fetch_pandas_all()
    lbi_trans = cur.execute(SQL_lbi).fetch_pandas_all()

    #print(replenishment)
    #print(forecast)
    #print(purchase_orders)
    #print(lbi_trans)

    replenishment=replenishment.to_numpy()
    forecast = forecast.to_numpy()
    purchase_orders=purchase_orders.to_numpy()
    lbi_trans = lbi_trans.to_numpy()

    current_pos = 0
    current_loc_ind = 0
    current_loc = 0
    sell_rates = []

    #link replenishment to weekly sell rate in sell rate list
    for i in replenishment:
        #print(i[4])
        if current_loc == 0:
            current_loc = replenishment[1]
            #print("first loc")
            #print(product[1])
            while locs[current_loc_ind] != current_loc and current_loc_ind < len(locs)-1:
                #loc_ID,p_ID,expiry[e][2],expiry[e][4],num_recieves,sell_rate,total_quantity,vendor
                data[current_pos][5].append(-1)
                current_pos = current_pos + 1
            sell_rates = data[current_pos][5]

        elif current_loc != product[1]:
            #print(current_loc_ind)
            data[current_loc_ind][5] = sell_rates
            current_loc = product[1]
            #print("after loc")
            #print(current_loc_ind)
            while current_loc_ind < len(locs)-1 and locs[current_loc_ind] != current_loc:
                print(current_loc_ind)
                #loc_ID,p_ID,expiry[e][2],expiry[e][4],num_recieves,sell_rate,total_quantity,vendor
                data[current_loc_ind][5].append(-1)
                current_loc_ind = current_loc_ind + 1
            sell_rates = data[current_pos][5]
        #sell_rates = data[0][5]
        date = i[5]
        year = date.strftime("%Y")
        week = date.strftime("%U")
        found = False
        #go through each sell_rate list entry to find what sell rate correlates to which replenishment entry based on week 
        while (found != True and current_pos != len(sell_rate)):
            drate = sell_rates[current_pos][0]
            if((drate.strftime("%U")== week) and (drate.strftime("%Y") == year)):
                #if week correlates is found append value
                sell_rates[current_pos].append(i[6])
                found = True
            else:
                #if week correlates is not found, append -1
                sell_rates[current_pos].append(-1)
            #done this way so never go back through sell rates/optimize
            current_pos= current_pos + 1

    # for case where there is no replenishment data, all entry is -1
    if len(replenishment) == 0:
        for l in locs: 
            for s in data[l][5]:
                s.append(-1)
    else:
        while current_pos != len(sell_rates):
            sell_rates[current_pos].append(-1)
            current_pos = current_pos + 1

    #each entry in sell rates look like: [date, weekly_sell_rate, avg_sell_rate, total_qty, repln link ]
    print("repln:")
    #print(len(replenishment))
    print(sell_rates[1])

    #coral will store forecasting link data
    coral = [-1] * len(sell_rates)
    #print(coral)

    #link raw demand forecast data to weekly sell rate in sell rate list
    for i in forecast:
        date = i[2]
        year = date.strftime("%Y")
        week = date.strftime("%U")
        found = False
        #print(year) range(len(sell_rate))
        #drate = sell_rates[current_pos][0]
        #if((drate.strftime("%U")== week) and (drate.strftime("%Y") == year
        #go through each sell rate to find correlating raw demand data-- optimization possible?
        for s in range(len(sell_rates)):
            drate = sell_rates[s][0]
            if ((drate.strftime("%U")==week) and (drate.strftime("%Y")== year) and coral[s] == -1):
                coral[s] = i[4]
            # print(coral[s])
                #print(coral)
                #print(i[4])
    #print(coral)
    #once all correlation found, append to sell rates
    for s in range(len(sell_rates)):
       # print(coral[s])
        sell_rates[s].append(coral[s])
    print("forecast:")
    print(sell_rates[1])

    #each entry in sell rates look like: [date, weekly_sell_rate, avg_sell_rate, total_qty, repln link, forecasting link ]

    #this code is to link each week with a purchase order if it is present-- needed?
    current_pos = 0
    for i in purchase_orders:
        date = i[2]
        year = date.strftime("%Y")
        week = date.strftime("%U")
        found = False
        #print(year) range(len(sell_rate))
        #append purchase order quanitity if found, -1 if nothing found
        while (found != True and current_pos < len(sell_rates)):
            #print(current_pos)
            drate = sell_rates[current_pos][0]
            if((drate.strftime("%U")== week) and (drate.strftime("%Y") == year)):
                sell_rates[current_pos].append(i[3])
                #print(i[3])
                found = True
            else:
                sell_rates[current_pos].append(-1)
            current_pos= current_pos + 1
    
    #if no purchase orders, entire row is -1
    if len(purchase_orders) == 0:
        for s in sell_rates:
            s.append(-1)
    else:
        #if less purchase orders then dates in sell_rate table, append -1 for remaining
        while current_pos != len(sell_rates):
            sell_rates[current_pos].append(-1)
            current_pos = current_pos + 1
    print("po:")
    print(sell_rates[1])

    #each entry in sell rates look like: [date, weekly_sell_rate, avg_sell_rate, total_qty, repln link,forecasting link, purchase order link ]

    #go through lbi_trans to find if week has correlating expiration date
    #coral will store all data
    coral = [None] * len(sell_rates)
    for i in lbi_trans:
        date = i[2]
        year = date.strftime("%Y")
        week = date.strftime("%U")
        #print(year) range(len(sell_rate))
        #current iteration will only use earliest expiration-- not all same
        #go through to see if there is correlating expiration date-- possible optimization
        for s in range(len(sell_rates)):
            drate = sell_rates[s][0]
            if ((drate.strftime("%U")==week) and (drate.strftime("%Y")== year) and coral[s] == None):
                coral[s] = i[6]
    
    #append -1 if expiration date not found
    for s in range(len(sell_rates)):
        #print(coral[s])
        if coral[s] == None:
            sell_rates[s].append(-1)
        else:
            sell_rates[s].append(coral[s])
    
    #each entry in sell rates look like: [date, weekly_sell_rate, avg_sell_rate, total_qty, repln link,forecasting link, purchase order link, expiration date]

    #identifing root cause

    print("lbi_trans")
    print(sell_rates[1])
    data[0][5] = sell_rates
    d = data[0]
    found = False
    total = 0
    root_cause = []
    #i = len(d[5])-1
    if len(d[5]) != 0:
        #print("in if")
        sell_rate = d[5]
        i = len(d[5])-1
        i_last_expiry = 0
        today = datetime.now()
        #find index of  earliest sell rate with expiration date after today
        while found != True and i > 0 and i < len(d[5]):
            #print(d)
            #print("split")
            #print(d[5][i])
            #print(i)
            if d[5][i][7] != -1:
                #print(d[5][i][7])
                if d[5][i][7] < today:
                    found = True
                    i_last_expiry = i + 1
            i = i -1
        if i_last_expiry > len(d[5])-1:
           i_last_expiry = len(d[5])-1

        #vendor D&E
        vendor_bool = False
        vi = 0
        #if there is a vendor damage event present within weeks, vendor damage event
        if  len(d[7]) != 0:
            while vi < len(d[7]) and vendor_bool == False:
            #for v in d[7]:
                #maybe check by week instead?
                v = d[7][vi]
                #print(i_last_expiry)
                #print(d[5])
                if d[5][i_last_expiry][0] < v[0]:
                    vendor_bool= True
                vi = vi+1
            if vendor_bool == True:
                root_cause.append("vendor D&E")
                all_rc[0] = all_rc[0] + 1

    #new product forecast
    #if num_recieves is less than 4 in weeks-- New Product Forecast (might need to be rechecked)
        if d[4] < 4:
            #print("new forecast")
            #print(d[5])
            avg = d[5][-1][2]
            if avg < 5:
                root_cause.append("New Product Forecast")
                all_rc[1] = all_rc[1] + 1
        i = i_last_expiry
        #both of following can have weekly breakdown
        found_of = False
        found_cp = False
        #if forecasted data is more than 2x the sell rate-- overforecasting
        while i != len(sell_rate) and found_of == False:
            #over forecasting
            #print(d[5][i])
            if d[5][i][4] > ((d[5][i][1])*2):
                #root_cause.append((d[5][i][0],"Over Forecasting",d[5][i][4] , d[5][i][1]*2))
                found_of = True
                all_rc[2] = all_rc[2] + 1
            i = i + 1
        i = i_last_expiry
        #if sell_rate/raw demand is much larger and is a multiple of case pack--possible optimization
        while i != len(sell_rate) and found_cp == False:
            #need to fix case pack
            if d[5][i][4] > ((d[5][i][1])*2):
                if (((d[5][i][4]) % d[3]) == 0):
                #root_cause.append("Case Pack")
                    found_cp = True
                    all_rc[3] = all_rc[3] + 1
            i = i + 1
        if found_of ==True:
           root_cause.append("Over Forecasting")
        if found_cp == True:
            root_cause.append("Case Pack")
    #print("after if")
    #if no other root cause discovered-- other
    if len(root_cause) == 0:
        root_cause.append("Other")
        all_rc[4] += 1
    final_report.append([d[1],d[0],root_cause])        
print(data)
#calculate percentage of each root cause entry
sum = 0
for i in all_rc:
    sum = sum + i

#print percentages
if sum != 0:
    print("Vendor D&E: " + str((all_rc[0]/sum)*100) +"%, New Product Forecast: " + str((all_rc[1]/sum)*100)+ "%, Overforecasting:" + str((all_rc[2]/sum)*100) + "%, Case Pack:" + str((all_rc[3]/sum)*100) + "%, Other:" + str((all_rc[4]/sum)*100))
#print product|location intersection with root cause
final_report = pd.DataFrame(final_report, columns=['Product_ID', 'Location_ID','Potential Root Cause'])
print(final_report)

        LOCATION_ID  PRODUCT_ID  SUM(EXPIRY_RISK_QUANTITY)  \
0               135          25                          0   
1                68          29                        -13   
2               100          29                        -45   
3               252          29                          0   
4               257          29                          0   
...             ...         ...                        ...   
200225          881      225208                          0   
200226         1025      225208                          0   
200227           30      225210                          0   
200228          621      225210                          0   
200229         1025      225210                          0   

       SUM(EXPIRY_RISK_COST)  MAX(RECENT_CASE_PACK)  PRODUCT_ID  LOCATION_ID  \
0                   0.000000                   10.0          25          135   
1                 -13.000000                   72.0          29           68   
2              

NameError: name 'loc_ID' is not defined